### 꿀리뷰
- 꿀리뷰 사이트에서 카테고리별 상품의 제품명, 전체평점, 최저가 데이터를 수집하세요.
- https://www.ggulreview.com/

In [81]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 카테고리 리스트 가져오기

In [82]:
url = "https://www.ggulreview.com/"
response = requests.get(url)
dom = BeautifulSoup(response.content, 'html.parser')

In [85]:
elements = dom.select(".active-categories-list__item")
len(elements)

20

In [87]:
element = elements[1]

In [90]:
key = element.select_one(".category-box__name").text
value = element.select_one("a").get("href").split('/p/')[1]
key, value

('무선 청소기', 'cordless-vacuums')

In [92]:
categories = {}
for element in elements:
    key = element.select_one(".category-box__name").text
    value = element.select_one("a").get("href").split('/p/')[1]
    categories[key] = value

categories

{'노트북': 'laptops',
 '무선 청소기': 'cordless-vacuums',
 '로봇 청소기': 'robotic-vacuums',
 '가습기': 'humidifiers',
 'TV': 'televisions',
 '무선 이어폰': 'wireless-earphones',
 '무선 스피커': 'wireless-speakers',
 '믹서기': 'blenders',
 '전기밥솥': 'rice-cookers',
 '에어프라이어': 'air-fryers',
 '공기청정기': 'air-purifiers',
 '건조기': 'dryers',
 '세탁기': 'washing-machines',
 '식기 세척기': 'dishwashers',
 '냉장고': 'refrigerators',
 '빔 프로젝터': 'projectors',
 '모니터': 'monitors',
 '에어컨': 'air-conditioners',
 '선풍기': 'fans',
 '제습기': 'dehumidifiers'}

# 크롬 브라우저 열고 페이지 이동하기

In [93]:
category = categories["TV"]
url = "https://www.ggulreview.com/p/{}".format(category)
driver = webdriver.Chrome()

# options = webdriver.ChromeOptions()
# options.add_argument("headless")
# driver = webdriver.Chrome(options=options)

driver.get(url)

# 상세 데이터 수집: 스크롤을 가장 아래로 이동

In [102]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# 상세 데이터 수집: 상품 더보기 클릭

In [103]:
try:
    driver.find_element_by_css_selector(".cta-button-type-Outline").click()
except:
    print("loaded all items")

loaded all items


# 상세 데이터 수집: 제품명, 전체평점, 최저가 데이터 수집

In [104]:
elements = driver.find_elements_by_css_selector(".product-list-item")
len(elements)

69

In [105]:
element = elements[0]

In [109]:
data1 = element.find_element_by_css_selector(".product-list-item__desc__name").text
data2 = element.find_element_by_css_selector(".product-list-item__desc__eval__score").text
data3 = element.find_element_by_css_selector(".product-list-item__desc__price").text
data1, data2, data3

('LG전자 울트라 HD TV AI 65인치', '4.7', '940,000원')

In [113]:
datas = []
for element in elements:
    datas.append({
        "data1" : element.find_element_by_css_selector(".product-list-item__desc__name").text,
        "data2" : element.find_element_by_css_selector(".product-list-item__desc__eval__score").text,
        "data3" : element.find_element_by_css_selector(".product-list-item__desc__price").text
    })
pd.DataFrame(datas)

,data1,data2,data3
0,LG전자 울트라 HD TV AI 65인치,4.7,"940,000원"
1,LG전자 올레드 갤러리 TV 55인치,4.4,"2,370,000원"
2,삼성전자 The Sero 43인치,4.5,"1,600,000원"
3,LG전자 올레드 AI 55인치,4.4,"1,210,000원"
4,삼성전자 The Serif 49인치,4.3,"1,670,000원"
...,...,...,...
64,와사비망고 몬스터빈 Artview 86인치,-,"1,850,000원"
65,삼성전자 The Frame 43인치,-,"1,380,000원"
66,삼성전자 QLED 8K 75인치,-,"6,430,000원"
67,와사비망고 WM UN650 스마트 AI 65인치,-,"590,000원"


In [114]:
class Ggulreview:
    
    # 생성자 함수
    def __init__(self, category="laptops"):
        url = "https://www.ggulreview.com/p/{}".format(category)
        self.driver = webdriver.Chrome()
        self.driver.get(url)
        self.category = category
        self.df = None

    # 카테고리 리스트 출력 메서드
    def category_list(self):
        url = "https://www.ggulreview.com/"
        response = requests.get(url)
        dom = BeautifulSoup(response.content, 'html.parser')
        elements = dom.select(".active-categories-list__item")
        categories = {}
        for element in elements:
            key = element.select_one(".category-box__name").text
            value = element.select_one("a").get("href").split('/p/')[1]
            categories[key] = value
        return categories
    
    # 카테고리 변경 메서드
    def change_category(self, category):
        url = "https://www.ggulreview.com/p/{}".format(category)
        self.driver.get(url)
        self.category = category
    
    # 더보기 이벤트 실행 메서드
    def __more(self):
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        self.driver.find_element_by_css_selector(".cta-button-type-Outline").click()
        time.sleep(1)
        
    # 상품 데이터 크롤링 메서드
    def crawling(self):
        try:
            while True:
                self.__more()
        except:
            print("loaded all items")
            
        elements = self.driver.find_elements_by_css_selector(".product-list-item")
        datas = []
        for element in elements:
            datas.append({
                "data1" : element.find_element_by_css_selector(".product-list-item__desc__name").text,
                "data2" : element.find_element_by_css_selector(".product-list-item__desc__eval__score").text,
                "data3" : element.find_element_by_css_selector(".product-list-item__desc__price").text
            })
        self.df = pd.DataFrame(datas)
        print("Done crawling")
    
    # 브라우져 종료 메서드
    def close(self):
        self.driver.quit()
        

In [115]:
# 객체 생성
gr = Ggulreview()

In [116]:
# 현재 카테고리
gr.category

'laptops'

In [75]:
# 카테고리 리스트 출력

In [117]:
gr.category_list()

{'노트북': 'laptops',
 '무선 청소기': 'cordless-vacuums',
 '로봇 청소기': 'robotic-vacuums',
 '가습기': 'humidifiers',
 'TV': 'televisions',
 '무선 이어폰': 'wireless-earphones',
 '무선 스피커': 'wireless-speakers',
 '믹서기': 'blenders',
 '전기밥솥': 'rice-cookers',
 '에어프라이어': 'air-fryers',
 '공기청정기': 'air-purifiers',
 '건조기': 'dryers',
 '세탁기': 'washing-machines',
 '식기 세척기': 'dishwashers',
 '냉장고': 'refrigerators',
 '빔 프로젝터': 'projectors',
 '모니터': 'monitors',
 '에어컨': 'air-conditioners',
 '선풍기': 'fans',
 '제습기': 'dehumidifiers'}

In [77]:
# 선풍기로 카테고리 변경

In [118]:
gr.change_category("fans")

In [119]:
gr.category

'fans'

In [80]:
# 데이터 수집

In [120]:
gr.crawling()

loaded all items
Done crawling


In [82]:
# 수집된 데이터 확인

In [121]:
gr.df.head()

,data1,data2,data3
0,발뮤다 그린팬S 무선,4.5,"370,000원"
1,한일전기 아기바람 ABFL-1470RTDC,4.2,"140,000원"
2,한일전기 아기바람 ABFL-1460RDC,4.2,"120,000원"
3,샤오미 스마트미 3세대 무선,4.0,"60,000원"
4,르젠 TR 입체회전,4.2,"70,000원"


In [ ]:
# 브라우져 닫기

In [123]:
gr.close()

KeyboardInterrupt: 

In [124]:
driver.quit()